In [32]:
from datetime import datetime, timezone

import pystac
import pystac_client
import planetary_computer
import rioxarray
from rasterio.warp import transform_bounds
from torchgeo.datasets.utils import BoundingBox

In [34]:
stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
##item_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20230415T154931_R054_T18TWR_20230416T011806"

# land-cover data from NRCan in Planetary-Computer
# examples:
#   - https://planetarycomputer.microsoft.com/api/stac/v1/collections/nrcan-landcover/items/CAN_LC_2015_CAL_13_15
col = ["nrcan-landcover"]
toi = [
    datetime(2015, 1, 1, tzinfo=timezone.utc),
    datetime(2020, 2, 1, tzinfo=timezone.utc),
]
# Québec, Canada - Surrounding Montréal and part of Vermont/New-Hampshire, USA
aoi_mtl_epsg3978 = [1600390, -285480, 1900420, 14550]
aoi_mtl_epsg4326 = transform_bounds("epsg:3978", "epsg:4326", *aoi_mtl_epsg3978)
# Québec, Québec City
aoi_qcc_epsg3978 = [1600390, 14550, 1900420, 314580]
aoi_qcc_epsg4326 = transform_bounds("epsg:3978", "epsg:4326", *aoi_qcc_epsg3978)
aoi = BoundingBox(*aoi_mtl_epsg4326, 0, 0) | BoundingBox(*aoi_qcc_epsg4326, 0, 0)
aoi = aoi[0:4]

print("COL", col)
print("AOI", aoi)
print("TOI", toi)

COL ['nrcan-landcover']
AOI [-75.07114805923383, 45.78709440860858, -70.27246530095542, 49.25983420104638]
TOI [datetime.datetime(2015, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2020, 2, 1, 0, 0, tzinfo=datetime.timezone.utc)]


In [39]:
catalog = pystac_client.Client.open(stac_url)
items = catalog.search(
    limit=10,
    bbox=aoi,
    datetime=toi,
    collections=col,
)
found_items = list(items.items())
found_items

[<Item id=CAN_LC_2015_CAL_14_15>,
 <Item id=CAN_LC_2015_CAL_14_14>,
 <Item id=CAN_LC_2015_CAL_13_15>,
 <Item id=CAN_LC_2015_CAL_13_14>,
 <Item id=CAN_LC_2015_CAL_12_15>,
 <Item id=CAN_LC_2015_CAL_12_14>]

In [51]:
classes = found_items[0].properties["label:classes"][0]["classes"]
cls_map = {i: label for i, label in enumerate(classes)}
cls_map

{0: 'Temperate or sub-polar needleleaf forest',
 1: 'Sub-polar taiga needleleaf forest',
 2: 'Temperate or sub-polar broadleaf deciduous forest',
 3: 'Mixed forest',
 4: 'Temperate or sub-polar shrubland',
 5: 'Temperate or sub-polar grassland',
 6: 'Sub-polar or polar shrubland-lichen-moss',
 7: 'Sub-polar or polar grassland-lichen-moss',
 8: 'Sub-polar or polar barren-lichen-moss',
 9: 'Wetland',
 10: 'Cropland',
 11: 'Barren lands',
 12: 'Urban',
 13: 'Water',
 14: 'Snow and Ice'}

In [42]:
signed_items = [
    planetary_computer.sign(item.assets["landcover"])
    for item in found_items
]

# Open one of the data assets
# (other asset keys to use: 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'preview')
asset_href = signed_items[0].href
ds = rioxarray.open_rasterio(asset_href)
ds

<xarray.DataArray (band: 1, y: 10001, x: 10001)>
[100020001 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 1.6e+06 1.6e+06 1.6e+06 ... 1.9e+06 1.9e+06 1.9e+06
  * y            (y) float64 1.454e+04 1.45e+04 ... -2.854e+05 -2.855e+05
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0